In [38]:
import numpy as np 
import pandas as pd 

In [39]:
df=pd.read_csv('AMZN_stock_data.csv')
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1997-05-15 00:00:00-04:00,0.121875,0.125000,0.096354,0.097917,1443120000,0.0,0.0
1,1997-05-16 00:00:00-04:00,0.098438,0.098958,0.085417,0.086458,294000000,0.0,0.0
2,1997-05-19 00:00:00-04:00,0.088021,0.088542,0.081250,0.085417,122136000,0.0,0.0
3,1997-05-20 00:00:00-04:00,0.086458,0.087500,0.081771,0.081771,109344000,0.0,0.0
4,1997-05-21 00:00:00-04:00,0.081771,0.082292,0.068750,0.071354,377064000,0.0,0.0
...,...,...,...,...,...,...,...,...
6982,2025-02-14 00:00:00-05:00,229.199997,229.889999,227.229996,228.679993,27031100,0.0,0.0
6983,2025-02-18 00:00:00-05:00,228.820007,229.300003,223.720001,226.649994,42975100,0.0,0.0
6984,2025-02-19 00:00:00-05:00,225.520004,226.830002,223.710007,226.630005,28566700,0.0,0.0
6985,2025-02-20 00:00:00-05:00,224.779999,225.130005,221.809998,222.880005,30001700,0.0,0.0


In [40]:
df['Date']= pd.to_datetime(df['Date'])

/var/folders/6w/jrhlkst96ql2zyhbfwzbz2lw0000gn/T/ipykernel_84653/2691600196.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date']= pd.to_datetime(df['Date'])


In [41]:
df['Close'].describe()

count    6987.000000
mean       40.685061
std        58.125169
min         0.069792
25%         2.110000
50%         9.021000
75%        73.006500
max       242.059998
Name: Close, dtype: float64

In [42]:
df['High'].describe()

count    6987.000000
mean       41.165619
std        58.787641
min         0.072396
25%         2.148250
50%         9.158500
75%        73.643497
max       242.520004
Name: High, dtype: float64

In [43]:
df['Close']=df['Close'].apply(lambda x: 1 if x>40.685061 else 0)
df['High']=df['High'].apply(lambda x: 1 if x>41.165619 else 0)


In [44]:
final=df[['Date','High','Close']]

In [45]:
final.set_index('Date')

,High,Close
Date,,
1997-05-15 00:00:00-04:00,0,0
1997-05-16 00:00:00-04:00,0,0
1997-05-19 00:00:00-04:00,0,0
1997-05-20 00:00:00-04:00,0,0
1997-05-21 00:00:00-04:00,0,0
...,...,...
2025-02-14 00:00:00-05:00,1,1
2025-02-18 00:00:00-05:00,1,1
2025-02-19 00:00:00-05:00,1,1


In [46]:
states = sorted(final['High'].unique())  # Assumes 'High' is categorical or discrete
state_to_index = {state: i for i, state in enumerate(states)}
num_states = len(states)

# Initialize transition counts
transition_counts = np.zeros((num_states, num_states))

# Count transitions from one 'High' value to the next
for t in range(1, len(final)):
    prev_state = final.iloc[t - 1]['High']
    curr_state = final.iloc[t]['High']
    i = state_to_index[prev_state]
    j = state_to_index[curr_state]
    transition_counts[i, j] += 1

# Normalize to get transition probabilities
transition_matrix = transition_counts / transition_counts.sum(axis=1, keepdims=True)

# Convert to DataFrame for readability
transition_df = pd.DataFrame(
    transition_matrix,
    index=states,
    columns=states
)

print(transition_df)

          0         1
0  0.999191  0.000809
1  0.001468  0.998532


Above you see a df that we are gonna use. We will use it to make an HMM model where our observations is the Close column and the Hidden state is the High column

Goal: We are going to predict a hidden state(some Si) given some observations () i.e. Forward Algorithm = P(si=1|O1,...Ot)

In [47]:
"Your task: You guys need to calculate the Emission Matrix and Transition Matrix as well as the initial forward inference"

'Your task: You guys need to calculate the Emission Matrix and Transition Matrix as well as the initial forward inference'

In [48]:
transition=None
emisssion=None

In [49]:
class HiddenMarkovModel:
    """A Hidden markov model which takes Transition matrix and Emmission matrix as inputs"""

    def __init__(self, transition_matrix, emission_matrix, prior=None):
      self.transition_matrix = transition_matrix
      self.emission_matrix = emission_matrix
      self.prior = prior
      return None

    def emission_dist(self, ev):
      if ev:
        return self.emission_matrix[0]
      else:
        return self.emission_matrix[1]
      return None

In [50]:
def forward_step(trans, fv, ev):
  emission_dist = emission_dist(ev)
  forward_inf = fv
  transition_matrix = trans

  nonNormSolution = emission_dist * np.array([[1,0],[0,1]]) @transition_matrix @ forward_inf
  alpha = nonNormSolution / np.sum(nonNormSolution)


  return alpha

In [51]:
def backward_step(trans, bv, ev):
  transition_matrix=trans
  backward_inference=bv
  emission_dist=emission_dist[ev]
  nonNormSol= transition_matrix @ np.diag(emission_dist)@ backward_inference.reshape(-1,1)
  return nonNormSol/np.sum(nonNormSol)


In [52]:
backward_inference = np.array([1.0, 1.0])
close_evdience = [True, True, False, True, True]
for ev in reversed(close_evdience):
  backward_inference = backward_step(transition, backward_inference, ev = ev) 
  print('Umbrella', ev, backward_inference)

UnboundLocalError: cannot access local variable 'emission_dist' where it is not associated with a value

In [ ]:
# Implement viterbi's algorithm here
def viterbi(HMM, evidence):
  t = len(evidence)
  evidence = evidence.copy()
  evidence.insert(0, None)

  m = [np.array([0,0]) for _ in range(len(evidence) - 1)]

  m[0] = forward_step(HMM, HMM.prior, evidence[1])

  for i in range(1, len(evidence) - 1):
    m[i] = np.multiply(HMM.emission_dist(evidence[i+1]),
     [max(np.multiply(HMM.transition_matrix[0], m[i-1])),
      max(np.multiply(HMM.transition_matrix[1], m[i-1]))])
  m = np.array(m)
  print(m)

  ml_probabilities = [True] * (len(evidence) -1)
  print(ml_probabilities)
  ml_path = 1 - np.argmax(m,axis=1)
  return ml_path, ml_probabilities, m